In [1]:
from pyspark.sql import *
from pyspark.sql.types import *
import re
import shutil
from time import time
from datetime import date
from GetPath import *
import pandas as pd
from functools import reduce

In [2]:
# Config
input_path='D:/csv_enhealth/Raw/cis_op_visiting_record/'
output_path='D:/Output/chenbingying/B1/'
icd_path='D:/Code/chenbingying/Data/ICD.csv'

In [3]:
# AGE字段清洗
# 将'XX周'、'XX月'、'XX天'、'XX小时'，换算为以岁为单位并向下取整
def Age(x,y):
    x=str(x).split('.')[0]
    if x=='None':
        return None
    xx=x
    for i in [[' ',''],['个',''],['岁',''],['一','1'],['二','2'],['三','3'],['四','4'],['五','5'],['六','6'],['七','7'],['八','8'],['九','9']]:
        x = x.replace(i[0],i[1])   
    if x =='':
        return None
    else:
        if re.findall('(\d+)',str(x))==[str(x)]:
            return x
        else:
            v = 0
            if '年' in x:
                s=x.split('年')
                try:
                    v += int(s[0])
                except:
                    return xx
                if len(s) == 2:
                    x = s[1]
                elif len(s) > 2:
                    return xx
            if '月' in x:
                s=x.split('月')
                try:
                    v += int(s[0])//12
                except:
                    return xx
                if len(s) == 2:
                    x = s[1]
                elif len(s) > 2:
                    return xx
            if '周' in x:
                s=x.split('周')
                try:
                    v += int(s[0])*7//365
                except:
                    return xx
                if len(s) == 2:
                    x = s[1]
                elif len(s) > 2:
                    return xx
            if '天' in x:
                s=x.split('天')
                try:
                    v += int(s[0])//365
                except:
                    return xx
                if len(s) == 2:
                    x = s[1]
                elif len(s) > 2:
                    return xx
    # 将取值小于0或大于(ETL_DATE-1900)的记录的年龄字段置空
            if y is not None:
                try:
                    if v<0 or v> y.year - 1900:
                        return None  
                    else:
                        return v
                except:
                    return None

In [4]:
# 性别处理
def Gender(x,y): # x为Gender,y为GenderName
    if(x is not None):
        if(x==2 or x=='2' or x=='F' or x=='2.0' or '女' in x):
            return '2'
        elif(x==1 or x=='1' or x=='M' or x=='1.0'  or '男' in x):
            return '1'
        else:
            if(y is not None):
                if(y==2 or y=='2' or y=='F' or y=='2.0' or '女' in y):
                    return '2'
                elif(y==1 or y=='1' or y=='M' or y=='1.0' or '男' in y):
                    return '1'
            else:
                return None        
    else:
        if(y is not None):
            if(y==2 or y=='2' or y=='F' or y=='2.0' or '女' in y):
                return '2'
            elif(y==1 or y=='1' or y=='M' or y=='1.0' or '男' in y):
                return '1'
            else:
                return None
        else:
            return None

In [5]:
# 性别名处理
def GenderName(x,y): # x为Gender，y为GenderName
    if(x is not None):
        if(x==2 or x=='2' or x=='F' or x=='2.0' or '女' in x):
            return '女'
        elif(x==1 or x=='1' or x=='M' or x=='1.0' or '男' in x):
            return '男'
        else:
            if(y is not None):
                if(y==2 or x=='2' or y=='F' or y=='2.0' or '女' in y):
                    return '女'
                elif(y==1 or y=='1' or y=='M' or y=='1.0'  or '男' in y):
                    return '男'
            else:
                return None        
    else:
        if(x is not None):
            if(x==2 or x=='2' or x=='F' or x=='2.0' or '女' in x):
                return '女'
            elif(x==1 or x=='1' or x=='M' or x=='1.0' or '男' in x):
                return '男'
            else:
                return None
        else:
            return None

In [6]:
# 去空格
def CutSpace(x):
    if(x is None):
        return None
    else:
        x = str(x).replace(' ','')
        if(x == ''):
            return None
        else:
            return str(x)

In [7]:
# 出生日期/就诊日期/发病日期/诊断日期
def Date(x,y):
    if(x is None or y is None):
        return None
    try: 
        if((y-x).days<0 or (x-date(1900,1,1)).days<0):
            return None
        else:
            return x
    except:
        return None

In [8]:
# 字符串日期转换
def DateFormat(x):
    if(x is None):
        return None
    numlist=[int(i) for i in re.findall(r"\d+",x)]
    if(len(numlist)<3):
        return None
    else:
        if(numlist[0]<1900 or numlist[1]<1 or numlist[1]>12 or numlist[2]<1 or numlist[2]>31):
            return None
        else:
            return date(numlist[0],numlist[1],numlist[2])

In [9]:
# 就诊类别名称填补
def Visiting_Type_Code(x,y):
    if(y is not None):
        return y
    else:
        if(x is None):
            return None
        x=str(x)
        if('.' in x):
            x=x.split('.')[0]
        x1=['100','101','102','103','200','300','400','401','500','600','601','999']
        x2=['普通门诊','专科门诊','专家门诊','特需门诊','急诊','急诊留观','普通住院','特需住院','家床','体检','职业康体检','其他']
        for i in range(len(x1)):
            if(x==x1[i]):
                return x2[i]
        return None

In [10]:
# 身份证类别名称填补
def Fill_ID_Type(x,y):
    if(y is not None):
        return y
    else:
        if(x is None):
            return None
        x=str(x)
        if('.' in x):
            x=x.split('.')[0]
        x1=['1','2','3','4','5','6','7','19','99']
        x2=['居民身份证','居民户口簿','护照','军官证','驾驶证','港澳居民来往内地通行证','台湾居民来往内地通行证','母亲身份证','其他法定有效证件']
        for i in range(len(x1)):
            if(x==x1[i]):
                return x2[i]
        return None

In [11]:
# 诊断代码清洗
def Code_Clean(x):
    if(x is None):
        return None
    if(len(x)<3):
        return None
    x=str(x).upper()
    if(x.isdigit() or x.isalpha()):
        return None
    for i in [[' ',''],['@,',''],['.-',''],['+','+;'],[',',';']]:
        x = x.replace(i[0],i[1])
    s='' 
    for i in x.split(';'):
        if(len(i)>2):
            s=s+i+';'
    if(s==''):
        return None
    else:
        return s[:-1]

In [12]:
# 诊断代码格式化
def Code_Map(x):
    if x is None:
        return None
    slist=x.split(';')
    ICDCode=''
    for i in slist:
        if(len(i)<3):
            continue
        if('M' in i and '/' in i):
            ICDCode=ICDCode+i[:8]+';'
        elif('*' in i):
            ICDCode=ICDCode+i[:3]+'*'+';'
        elif(i[:3] in ICD):
            ICDCode=ICDCode+i[:3]+';'
    if(ICDCode==''):
        return None
    else:
        return ICDCode[:-1]

In [13]:
# 诊断名称格式化
def Name_Map(x):
    if x is None:
        return None
    for i in ['.','。','、',';','；','?','？','，',',',':','：',' ']:
        x=x.replace(i,';')
    for i in('\(.*\)','（.*\)','\(.*）','（.*）','\[.*\]'):
        x=re.sub(i,'',x)
    ICDName=''
    for i in x.split(';'):
        if(len(i)>1):
            ICDName=ICDName+i+';'
    if(ICDName==''):
        return None
    else:
        return ICDName[:-1]

In [14]:
# ICD编码
def ICD_Code(x,y): # x为Code，y为Name
    if(x is None and y is None):
        return None
    if(x is not None):
        return x
    if(y is not None):
        ICDCode=''
        y=str(y)
        ICDName=re.split('[;\d]',y)
        for i in ICDName:
            ICD=ICDDict.get(i,None)
            if(ICD is not None):
                ICDCode=ICDCode+ICD+';'
        if(ICDCode==''):
            return None
        else:
            return ICDCode[:-1]

In [15]:
def Build(filelist,output_path,name):
    t_start=time()
    start = time()
    files=[]
    spark = SparkSession.builder.appName('B1_Create').config('spark.dynamicAllocation.enabled','true').config('spark.debug.maxToStringFields', '100').config('spark.sql.execution.arrow.enable','true').config('spark.executor.memory','48g').config('spark.driver.memory', '48g').config('spark.core.connection.ack.wait.timeout','36000s').config('spark.executor.heartbeatInterval','36000s').config('spark.network.timeout', '50000s').config('spark.rpc.lookupTimeout', '5000s').config('spark.shuffle.io.connectionTimeout', '50000s').config("spark.sql.broadcastTimeout","50000s").config("spark.network.timeout","50000s").getOrCreate()
    # 读入数据
    print('开始处理分块文件')
    for file in filelist:
        files.append(spark.read.option("inferSchema","true").option("delimiter", "\t").option("header","false").csv(file))
    reduce(lambda x,y:x.unionByName(y), files).createOrReplaceTempView('B1_Raw')
    print('读入数据完成耗时：%.2f秒'%(time()-start))
    start = time()
    spark.udf.register('Age',Age,StringType())
    spark.udf.register('Gender',Gender,StringType())
    spark.udf.register('GenderName',GenderName,StringType())
    spark.udf.register('CutSpace',CutSpace,StringType())
    spark.udf.register('Date',Date,DateType())
    spark.udf.register('DateFormat',DateFormat,DateType())
    spark.udf.register('Visiting_Type_Code',Visiting_Type_Code,StringType())
    spark.udf.register('Fill_ID_Type',Fill_ID_Type,StringType())
    spark.udf.register('Code_Clean',Code_Clean,StringType())
    spark.udf.register('Code_Map',Code_Map,StringType())
    spark.udf.register('Name_Map',Name_Map,StringType())
    spark.udf.register('ICD_Code',ICD_Code,StringType())
    # 清洗数据
    spark.sql("""
    select
    CutSpace(_c0) as MEDICAL_INSTITUTION_CODE
    ,CutSpace(_c1) as MEDICAL_INSTITUTION_NAME
    ,CutSpace(_c2) as VISITING_SERIAL_NUMBER
    ,CutSpace(_c3) as VISITING_TYPE_CODE
    ,Visiting_Type_Code(CutSpace(_c3),CutSpace(_c4)) as VISITING_TYPE_NAME
    ,CutSpace(_c5) as FILE_NUMBER
    ,CutSpace(_c6) as HEALTH_CARD_NUMBER
    ,CutSpace(_c7) as CARD_NUMBER
    ,CutSpace(_c8) as CARD_TYPE
    ,CutSpace(_c9) as CARD_TYPE_NAME
    ,CutSpace(_c10) as ID_NUMBER
    ,CutSpace(_c11) as ID_TYPE
    ,Fill_ID_Type(CutSpace(_c11),CutSpace(_c12)) as ID_TYPE_NAME
    ,CutSpace(_c13) as NAME
    ,Date(to_date(_c14),to_date(_c31)) as BIRTH_DATE
    ,Age(_c15,_c31) as AGE
    ,Gender(CutSpace(_c16),CutSpace(_c17)) as GENDER
    ,GenderName(CutSpace(_c16),CutSpace(_c17)) as GENDER_NAME
    ,CutSpace(_c18) as VISITING_SIGN
    ,CutSpace(_c19) as REFERRAL_SIGN
    ,CutSpace(_c20) as VISITING_DEPARTMENT_CODE
    ,CutSpace(_c21) as VISITING_DEPARTMENT_NAME
    ,Date(DateFormat(cast (_c22 as string)),to_date(_c31)) as VISITING_DATE
    ,CutSpace(_c23) as OUTPATIENT_SYMPTOM_NAME
    ,_c24 as SYMPTOM_DIAGNOSTIC_CODE
    ,Date(to_date(_c25),to_date(_c31)) as DATE_OF_ONSET
    ,_c26 as TYPE_OF_DIAGNOSIS_CODE
    ,Code_Clean(_c27) as DISEASE_DIAGNOSIS_CODE
    ,Name_Map(_c28) as NAME_OF_DISEASE_DIAGNOSIS
    ,_c29 as DIAGNOSIS_DESCRIPTION
    ,Date(to_date(_c14),to_date(_c31)) as DATE_OF_DIAGNOSIS
    ,ICD_Code(Code_Map(Code_Clean(_c27)),Name_Map(_c28)) as ICD_CODE
    ,_c31 as ETL_DATE
    ,_c32 as AREA
    ,_c33 as id_birth_date
    ,_c34 as id_gender
    from B1_Raw""").coalesce(1).write.csv(output_path+name,mode='overwrite',header=True)
    print('关键字段清洗耗时：%.2f秒'%(time()-start))
    # 释放资源
    spark.catalog.dropTempView('B1_Raw')
    shutil.move(GetPath(output_path+name+'/','csv'),output_path+name+'.csv')
    shutil.rmtree(output_path+name)
    print('清洗数据总耗时：%.2f秒'%(time()-t_start))
    print(f'已输出至：{output_path}{name}.csv')


In [16]:
def Create_B1(input_path,output_path,icd_path):
    t_start=time()
    filelist=GetPath(input_path,'csv')
    global ICD, ICDDict
    ICDCode= pd.read_csv(icd_path)
    ICD=list(set(ICDCode['ICD_CODE'].to_list()))
    ICDDict=dict(zip(ICDCode['ICD_NAME'].to_list(),ICDCode['ICD_CODE'].to_list()))
    Build(filelist[101:],output_path,'tb_cis_op_visiting_record_202201_202205')
    Build(filelist[:54],output_path,'tb_cis_op_visiting_record_201401_201806')
    Build(filelist[54:71],output_path,'tb_cis_op_visiting_record_201807_201912')
    Build(filelist[71:88],output_path,'tb_cis_op_visiting_record_202001_202012')
    Build(filelist[88:101],output_path,'tb_cis_op_visiting_record_202101_202112')
    print('总耗时：%.2f秒'%(time()-t_start))

In [17]:
if __name__=="__main__":
    Create_B1(input_path,output_path,icd_path)

开始处理分块文件
读入数据完成耗时：58.99秒
